In [ ]:
# Publications
import os
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configuration des chemins
chrome_driver_path = r"....."
chrome_binary_path = r"....."
download_folder = r"....."
progress_file = "progress.json"

def save_progress(done_issues, done_articles):
    with open(progress_file, "w") as f: # Ouverture d'un fichier en mode écriture avec les caractéristiques suivantes, écrase son contenu si il existe
        json.dump({"done_issues": done_issues, "done_articles": done_articles}, f) # type - json ; avec deux clès

def load_progress():
    if os.path.exists(progress_file): # Si progress file existe ; 
        with open(progress_file, "r") as f: # Ouvre progress file en mode lecture
            return json.load(f) # Charge le fichier en dictionnaire python
    return {"done_issues": [], "done_articles": []} # Retourne deux listes

def setup_driver():
    """Configure et lance WebDriver."""
    options = Options() # Création d'un objet option dans lequel ont va mettre les options dites "classiques".
    options.add_argument("--headless") # Lance le navigateur sans interface graphique, pas besoin.
    options.add_argument("--disable-gpu") # Désactive l'accéleration GPU, pas besoin.
    options.add_argument("--disable-dev-shm-usage") # Force l'utilisation du disque.
    options.add_argument("--no-sandbox") # Désactive le sandboxing, pas besoin.
    options.binary_location = chrome_binary_path # Défini le chemin d'utilisation de chromium, pas envie de le définir comme navigateur principal.
    # Création d'un dictionnaire contenant les options expérimentales
    prefs = {"download.default_directory": download_folder, # Spécifie que le répertoire pour le téléchargement est le répertoire téléchargement préalablement défini
             "download.prompt_for_download": False, # Désactiver la boite de confirmation pour les téléchargements, c'est pas pratique en mode headless et j'ai pas envie de cliquer 10000 fois.
             "download.directory_upgrade": True, # Migre ancien répertoire vers nouveau sans demander de comfirmation. 
             "safebrowsing.enabled": True} # Activer la fonction de navigation sécurisée : élémentaire, my dear Watson. 
    options.add_experimental_option("prefs", prefs) # Utilise la fonction .add_experimental_option pour associer la clè "prefs' aux valeurs du dictionnaire prefs

    service = ChromeService(executable_path=chrome_driver_path) # Création d'un objet service avec le chemin éxécutable prélablement défini. 
    return webdriver.Chrome(service=service, options=options) # service = service ; options = options ; here we go.. 

def get_links(driver, url, xpath): # Fonction qui ouvre un URL par le driver et navigue sur la page pour récupérer d'autres URL correspondant à un XPATH donné.
    driver.get(url) # Ouvre un URL
    time.sleep(3.5)  # Temps d'attente en seconde, le temps que la page s'ouvre
    return [link.get_attribute("href") for link in driver.find_elements(By.XPATH, xpath)] # Compréhension de liste : récupérer l'attribut href, pour chaque lien, dans le driver, trouver l'élément XPATH spécifié

def wait_for_download(timeout=45, min_size=10_000): # Fonction qui attend que le fichier soit téléchargé.
    initial_files = set(os.listdir(download_folder)) # Liste les fichiers présents dans le répertoire de téléchargements
    elapsed = 0 # Initialise un compteur de temps
    while elapsed < timeout: # Tant que le compteur de temps est inférieur au timeout, soit 45 s.
        new_files = set(os.listdir(download_folder)) - initial_files # Défini objet "nouveaux fichiers" qui a ce qu'il y a dans le répértoire moins intial_files.
        for file in new_files: # Pour chaque fichier dans "nouveaux fichiers"
            file_path = os.path.join(download_folder, file) # Combine le nom de répertoire et le nom de fichier pour créer un chemin.
            if file.endswith(".pdf") and os.path.getsize(file_path) > min_size: # Si le fichier est un pdf et qu'il est supérieur à la taille minimum requise, 
                return file_path # Retourne le chemin
        time.sleep(1.4)  # Attendre 1,4 seconde avant de revérifier les fichiers
        elapsed += 1.4 # + 1,4 s de temps écoulé
    return None # Fin de la fonction

def download_pdf(driver, article_url, done_articles):
    if article_url in done_articles:# : Vérifie si il est déjà listé pour éviter les doublons
        print(f"✔️ Déjà téléchargé : {article_url}") # Si oui, log.
        return
    driver.get(article_url) # Si non, ouvre la page web de l'article
    time.sleep(3.5) # Attend 3,5 s. que la page soit bien ouverte.
    try:
        WebDriverWait(driver, 14).until( 
            EC.element_to_be_clickable((By.XPATH, "//div[@onclick='pdfDownloadAction()']")) # Utilise WDwait jusqu'à ce qu'apparaisse un élément cliquable défini par le XPATH mentionné ; attend au maximul 14 seconde. 
        ).click() # puis clique sur l'élément.
        pdf_path = wait_for_download() # Appel la fonction wait_for_download pour attendre que le fichier soit téléchargé
        if pdf_path: # Au terme de la fonction "attendre le téléchargement", si un pdf_path a été créé, alors :
            print(f"📂 PDF téléchargé : {pdf_path}") # log :
            done_articles.append(article_url) # Ajouter l'URL de l'article à "articles faits":
            save_progress(done_issues, done_articles) # Utiliser la fonction "sauvegarder la progression" sur les valeurs définies
    except Exception as e: # Si pas de pdf_path
        print(f"⚠️ Erreur téléchargement : {e}") # log

def main():
    os.makedirs(download_folder, exist_ok=True) # Créer le fichier de téléchargement si il n'existe pas déjà
    progress = load_progress() # Créer un objet qui contient le résultat de load_progress
    done_issues, done_articles = progress["done_issues"], progress["done_articles"] # Créer deux objets qui sont des listes de ce qui a déjà été fait
    
    max_retries, retry_count = 15, 0 # Il faut parfois relancer le prog, dont mettre en place un compteur.
    while retry_count < max_retries: # Tant que le programme a été relancé moins de 15 fois, on recommence
        try:
            driver = setup_driver() # Lancement du webdriver
            base_url = "....." # Définition de l'URL de base
            issue_links = get_links(driver, base_url, "//a[starts-with(@href, '.....')]") # Récupération des liens des éditions par la fonction get_links
            # On cherche des balises html...
            for issue_url in issue_links: # Pour chaque url dans les liens des éditions on va directement après le continue
                if issue_url in done_issues: # Si une édition est dans les éditions traitées
                    print(f"✔️ Édition déjà traitée : {issue_url}") # log
                    continue
                article_links = get_links(driver, issue_url, "//a[starts-with(@href, '.....')]") # Récupération des liens des article par la fonction get_links
            # On cherche des balises html...
                for article_url in article_links: # Pour chaque url dans les liens des articles 
                    download_pdf(driver, article_url, done_articles) # On appel la fonction download pdf
                done_issues.append(issue_url) #Quand une édition est finie, on la rajoute aux éditions réalisées.
                save_progress(done_issues, done_articles) # On sauvegarde les progès.
            # Au revoir !
            driver.quit()
            print("🚀 Script terminé avec succès !")
            break
            # Oh non !
        except Exception as e:
            print(f"⚠️ Crash détecté : {e}")
            retry_count += 1
            print(f"🔁 Redémarrage ({retry_count}/{max_retries})...")
            if retry_count == max_retries:
                print("❌ Échec après plusieurs tentatives. Arrêt.")

if __name__ == "__main__": # Au cas où. 
    main()